In [118]:
import psycopg2
import pandas as pd
import sh
from IPython.display import display, HTML, Javascript

# Note: assumes that you have a "parcels"/"nfirs"/"firecares-vm" service listed in your ~/.pg_service.conf, "firecares-vm"
# should point at your dev FireCARES vm's database (which requires the FireDepartment.owned_tracts_geom property for
# correct intersection), "parcels" and "nfirs" connections point to the production database instances

parcels = psycopg2.connect('service=parcels')
fc = psycopg2.connect('service=firecares-vm')
nfirs = psycopg2.connect('service=nfirs')

def heading(text, tag='h3'):
    display(HTML('<{tag}>{text}</{tag}>'.format(tag=tag, text=text)))

<IPython.core.display.Javascript object>

In [ ]:
# Create an intermediary table for backing-up only the department information we care about for local restoration
# into the parcels database
with fc.cursor() as c:
    c.execute('SELECT id, owned_tracts_geom INTO firedepartment_owned_geom FROM firestation_firedepartment where owned_tracts_geom IS NOT null;')
    fc.commit()
sh.pg_dump('service=firecares-vm', '-O', '-t', 'firedepartment_owned_geom', _out='/tmp/firedepartment_owned_geom.sql')

In [ ]:
# Might be overkill, but filtering down to a much smaller table size to scan when performing intersections
# this will definitely take some time to complete
with parcels.cursor() as c:
    c.execute("""SELECT ogc_fid, wkb_geometry, parcel_id, state, zip, land_use, story_nbr INTO parcel_stories FROM parcels WHERE
story_nbr IS NOT null AND land_use NOT IN ('163', '112');""")
    c.execute("""CREATE INDEX ON parcel_stories USING gist (wkb_geometry);""")
    parcels.commit()

In [ ]:
# Load FDs
sh.psql('service=parcels', _in=file('/tmp/firedepartment_owned_geom.sql', 'r'))

In [80]:
df = pd.read_sql_query("""SELECT count(1) FROM parcels p INNER JOIN "LUSE_swg" u ON u."Code" = p.land_use AND p.state = 'DC';""", parcels)
print 'Total parcels in DC w/ land_use hit: {}'.format(df['count'][0])

df = pd.read_sql_query("""SELECT count(1) FROM parcels p WHERE p.land_use is null and p.state = 'DC';""", parcels)
print 'Total parcels in DC w/ no land_use: {}'.format(df['count'][0])

Total parcels in DC w/ land_use hit: 133014
Total parcels in DC w/ no land_use: 3538


In [128]:
# Get counts over DC for medium/high structures
df = pd.read_sql_query("""select count(1), p.story_nbr, p.land_use, lu.risk_category, lu."Description", lu.residential
from parcels p
    left join "LUSE_swg" lu
    on lu."Code" = p.land_use
where state = 'DC' and lu.risk_category in ('Medium', 'High')
group by p.story_nbr, p.land_use, lu.risk_category, lu."Description", lu.residential
order by count desc, p.story_nbr, p.land_use""", parcels)

df.style.highlight_null(null_color='red')

In [130]:
# Get counts over Missouri for medium/high structures

pd.read_sql_query("""select count(1), p.story_nbr, p.land_use, lu.risk_category, lu."Description", lu.residential
from parcels p
    left join "LUSE_swg" lu
    on lu."Code" = p.land_use
where state = 'MO' and lu.risk_category in ('Medium', 'High')
group by p.story_nbr, p.land_use, lu.risk_category, lu."Description", lu.residential
order by count desc, p.story_nbr, p.land_use""", parcels)

df.style.highlight_null(null_color='red')

In [131]:
import numpy as np
import matplotlib.pyplot as plt

In [135]:
plt.plot(df['count'])

In [136]:
%load_ext sql

/usr/local/lib/python2.7/site-packages/IPython/config.py:13: ShimWarning: The `IPython.config` package has been deprecated. You should import from traitlets.config instead.
  "You should import from traitlets.config instead.", ShimWarning)
/usr/local/lib/python2.7/site-packages/IPython/utils/traitlets.py:5: UserWarning: IPython.utils.traitlets has moved to a top-level traitlets package.
  warn("IPython.utils.traitlets has moved to a top-level traitlets package.")


In [143]:
%sql postgresql:///?service=nfirs

u'Connected: None@'

In [145]:
%%sql
select count(1) from parcels where state = 'DC';

1 rows affected.


count
136552
